In [6]:
import pandas as pd
import json

# Load Metadata
metadata = pd.read_csv('Metadata.csv')

# Remove rows where 'Country_Code' is null from metadata
metadata = metadata.dropna(subset=['Country_Code'])

# Define a Mapping for File Information with new column names and CSV names
file_info = {
    "API_EG.ELC.ACCS.RU.ZS_DS2_en_csv_v2_4364070.json": {"new_column": "ACCSRU", "csv_name": "access_to_rural_population.csv"},
    "API_EG.ELC.ACCS.ZS_DS2_en_csv_v2_4353549.json": {"new_column": "ACCSTO", "csv_name": "access_to_electricity_total.csv"},
    "API_EG.ELC.ACCS.UR.ZS_DS2_en_csv_v2_4367536.json": {"new_column": "ACCSUR", "csv_name": "access_to_urban_population.csv"},
    "API_EG.ELC.LOSS.ZS_DS2_en_csv_v2_4357259.json": {"new_column": "ELECLOSS", "csv_name": "electricity_loss.csv"},
    "API_EG.ELC.NUCL.ZS_DS2_en_csv_v2_4357319.json": {"new_column": "ELECNUC", "csv_name": "electricity_from_nuclear.csv"},
    "API_EG.ELC.PETR.ZS_DS2_en_csv_v2_4356435.json": {"new_column": "ELECOIL", "csv_name": "electricity_from_oil.csv"},
    "API_EG.ELC.RNWX.KH_DS2_en_csv_v2_4356007.json": {"new_column": "ELECRENEW", "csv_name": "electricity_from_renewable.csv"}
}

# Load, Clean, and Store Each DataFrame
for file_name, info in file_info.items():
    with open(file_name, 'r') as file:
        data = json.load(file)
    
    # Convert JSON data to DataFrame
    df = pd.DataFrame(data)
    
    # Remove the first row (index 0) and use the next row as column headers
    df = df.drop(index=0)
    df.columns = df.iloc[0]
    df = df.drop(index=df.index[0])
    df = df.reset_index(drop=True)
    
    # Melt the DataFrame to transform it from wide to long format
    melted_df = df.melt(
        id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'],
        var_name='Year',
        value_name=info['new_column']
    )
    
    # Convert 'Year' to numeric and remove rows where 'Year' is null or <= 1999
    melted_df['Year'] = pd.to_numeric(melted_df['Year'], errors='coerce')
    melted_df = melted_df.dropna(subset=['Year'])
    melted_df = melted_df[melted_df['Year'] > 1999]
    
    # Remove null values from the value column
    melted_df = melted_df.dropna(subset=[info['new_column']])
    
    # Merge Metadata to add Region and IncomeGroup
    melted_df = pd.merge(
        melted_df,
        metadata,
        left_on='Country Code',
        right_on='Country_Code',
        how='left'
    )
    
    # Remove rows where 'Region' or 'IncomeGroup' is null
    melted_df = melted_df.dropna(subset=['Region', 'IncomeGroup'])
    
    # Reorder columns to place the value column at the end
    cols = [col for col in melted_df.columns if col != info['new_column']]
    cols.append(info['new_column'])
    melted_df = melted_df[cols]
    
    # Save the melted DataFrame to CSV
    melted_df.to_csv(info['csv_name'], index=False)

print("Data processing complete. Individual CSV files have been created.")


Data processing complete. Individual CSV files have been created.
